In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix,accuracy_score
import tensorflow as tf


2024-03-01 01:01:12.115128: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 01:01:12.115276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 01:01:12.250067: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
def process_data(img_dims, batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=.3, vertical_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        directory='/kaggle/input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(img_dims, img_dims),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    
    test_generator = test_datagen.flow_from_directory(
        directory='/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(img_dims, img_dims),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    
    testData = []
    testLabels = []
    
    for condition in ['NORMAL', 'PNEUMONIA']:
        for img_name in os.listdir('/kaggle/input/chest-xray-pneumonia/chest_xray/test/' + condition):
            img_path = os.path.join('/kaggle/input/chest-xray-pneumonia/chest_xray/test/', condition, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_dims, img_dims))
            img = img.astype('float32') / 255
            testData.append(img)
            if condition == 'NORMAL':
                label = 0
            elif condition == 'PNEUMONIA':
                label = 1
            testLabels.append(label)

    testData = np.array(testData)
    testLabels = np.array(testLabels)
    
    return train_generator, test_generator, testData, testLabels


In [8]:
# Define input shape
input_shape = (150, 150, 3)

# Process the data
train_generator, test_generator, testData, testLabels = process_data(150, 8)

# Define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

# Train the model
num_epochs = 10
verbose = 1
steps_per_epoch = len(train_generator)
validation_steps = len(test_generator)

history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=verbose, validation_data=test_generator, validation_steps=validation_steps)

# Evaluate the model
pred = model.predict(testData)
predicted_labels = np.round(pred)
accuracy = accuracy_score(testLabels, predicted_labels) * 100

print("Accuracy: {:.2f}%".format(accuracy))

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


652/652 ━━━━━━━━━━━━━━━━━━━━ 157s 238ms/step - accuracy: 0.7456 - loss: 0.7772 - val_accuracy: 0.7917 - val_loss: 0.4295
Epoch 2/10


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


652/652 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 196s 236ms/step - accuracy: 0.8430 - loss: 0.3675 - val_accuracy: 0.7628 - val_loss: 0.4646
Epoch 4/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 155s 237ms/step - accuracy: 0.8783 - loss: 0.2895 - val_accuracy: 0.8349 - val_loss: 0.3900
Epoch 6/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 0s 23us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 202s 237ms/step - accuracy: 0.8799 - loss: 0.2731 - val_accuracy: 0.7660 - val_loss: 0.5365
Epoch 8/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 0s 19us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
652/652 ━━━━━━━━━━━━━━━━━━